
## Ask a Legal Research Assistant Bot about the US Constitution

This Bot should do the following:

* Read the US_Constitution.txt file inside the some_data folder
* Split this into chunks (you choose the size)
* Write this to a ChromaDB Vector Store
* Use Context Compression to return the relevant portion of the document to the question

In [ ]:
!pip install openai
!pip install --upgrade langchain
!pip install wikipedia
!pip install chromadb
!pip install tiktoken

In [8]:
# Build a sample vectorDB
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
import os


In [5]:
def us_constitution_helper(question):
    '''
    Takes in a question about the US Constitution and returns the most relevant
    part of the constitution. Notice it may not directly answer the actual question!

    Follow the steps below to fill out this function:
    '''
    # PART ONE:
    # LOAD "some_data/US_Constitution in a Document object
    loader = TextLoader("sample_data/US_Constitution.txt")
    documents = loader.load()

     # PART TWO
     # Split the document into chunks (you choose how and what size)
    splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
    docs = splitter.split_documents(documents)

     # PART THREE
     # EMBED THE Documents (now in chunks) to a persisted Chroma
    embedding = OpenAIEmbeddings()
    db = Chroma.from_documents(docs, embedding, persist_directory="./legal_db")

    # PART FOUR
    # Use ChatOpenAI and ContextualCompressionRetriever to return the most
    # relevant part of the documents.
    chatllm = ChatOpenAI()
    compressor = LLMChainExtractor.from_llm(llm=chatllm)
    context_retrieve_compressor = ContextualCompressionRetriever(base_compressor= compressor,
    base_retriever=db.as_retriever())

    # PART FIVE
    # Use Context Compression to return the relevant portion of the document to the question
    # (you choose the size)
    result = context_retrieve_compressor.get_relevant_documents(question)
    return result[0].page_content


## Example Usage:

Notice how it doesn't return an entire Document of a large chunk size, but instead the "compressed" version!

In [ ]:
print(us_constitution_helper("What is the 13th Amendment?"))

13th Amendment
Section 1
Neither slavery nor involuntary servitude, except as a punishment for crime whereof the party shall have been duly convicted, shall exist within the United States, or any place subject to their jurisdiction.
